In [71]:
#Michelle Brenes
#Laser Rivera
#Julio Marin 
#Mohnish Hemdev
#Jayson Morán

In [72]:
#pip install pandas
#pip install dash
#pip install geopandas
#pip install pipwin
#pip install fiona
#pip install wheel

In [73]:
import dash
import dash_core_components as dcc 
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input,Output
import matplotlib.pyplot as plt
from urllib.request import urlopen
import geopandas as gpd

In [74]:
#Importe del json que contiene al mapa de panama
path = 'https://raw.githubusercontent.com/JaysonAMR/Semestral_Topicos/main/Json/PAN_ADM01_PROV_V01.topo.json'
geo_pan = gpd.read_file(path)

#Importe del csv con los datos del movimiento de contenedores de los puertos
df = pd.read_csv('https://raw.githubusercontent.com/JaysonAMR/Semestral_Topicos/main/Resumido-por-a%C3%B1os.csv')

In [75]:
geo_pan.head()

,id,COD_PROV,PROV_NAME,ISO_CODE,geometry
0,None,01,Bocas del Toro,PA-1,"MULTIPOLYGON (((-82.00026 8.95718, -82.01260 8..."
1,None,04,Chiriquí,PA-4,"MULTIPOLYGON (((-82.44513 7.87112, -82.44337 7..."
2,None,02,Coclé,PA-2,"POLYGON ((-80.13616 8.96596, -80.14497 8.94522..."
3,None,03,Colón,PA-3,"MULTIPOLYGON (((-79.85114 9.39667, -79.84644 9..."
4,None,05,Darién,PA-5,"MULTIPOLYGON (((-78.01407 8.23574, -77.99350 8..."


In [76]:
df.head()

,COD_PROV,PROV_NAME,Puerto,Año,Trasbordo,Local,Zona Libre
0,1,Bocas del Toro,Bocas Fruit Co.,2017,0,16488,0
1,1,Bocas del Toro,Bocas Fruit Co.,2018,0,22896,0
2,1,Bocas del Toro,Bocas Fruit Co.,2019,0,58534,0
3,1,Bocas del Toro,Bocas Fruit Co.,2020,12362,83400,0
4,1,Bocas del Toro,Bocas Fruit Co.,2021,53958,63834,0


In [77]:
app = dash.Dash(__name__)

In [78]:
app.layout = html.Div([
    
    html.Div([
        html.H1('Movimientos de contenedores en el SPN'),
        html.Img(src='assets/contenedor.png')
    ], className = 'banner'),

    html.Div([
        html.Div([
            html.P('Selecciona el tipo de transito', className= 'flix_label', style={'color':'black', 'margin-top': '2px'}),
            dcc.RadioItems( id= 'año-radioitem',
                            labelStyle={'display': 'inline-block'},
                            options=[
                                {'label' : 'Local', 'value' : 'Local'},
                                {'label' : 'Trasbordo', 'value' : 'Trasbordo'},
                                {'label' : 'Zona Libre', 'value' : 'Zona Libre'}
                            ], value= 'Año',
                            style= {'text-aling' : 'center' , 'color' : 'black'}, className= 'dcc_compon'),
        ], className= 'create_container2 five columns', style= {'margin-bottom': '20px'}),
        
    ], className = 'row flex-display'),

    html.Div([
        html.Div([
            dcc.Graph(id = 'my_graph', figure = {})
        ], className= 'create_container2 eight colums'), 

        html.Div([
            dcc.Graph(id = 'mpie_graph', figure = {})
        ], className= 'create_container2 five colums'), 
    ], className='row flex-display'),

],id='mainContainer', style= {'display': 'flex', 'flex-direction': 'column'})

In [79]:
@app.callback (
    Output('my_graph', component_property= 'figure'),
    [Input('año-radioitem', component_property='value')]
)

def update_graph(value):
    if value == 'Local':
        fig = px.bar(
            data_frame=df(2017),
            x = 'Puerto',
            y = 'Local'
            
        )
    elif value == 'Trasbordo':
        fig = px.bar(
            data_frame=df,
            x = 'Puerto',
            y = 'Trasbordo'
        )
    else:
        fig = px.bar(
            data_frame=df,
            x = 'Puerto',
            y = 'Zona Libre'
        )
    return fig

@app.callback (
    Output('mpie_graph', component_property= 'figure'),
    [Input('año-radioitem', component_property='value')]
)
def update_graph(value):
    if value == 'Local':
        fig = px.choropleth(data_frame=df, 
                    geojson=geo_pan, 
                    locations='PROV_NAME', # nombre de la columna del Dataframe
                    featureidkey='properties.PROV_NAME',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                    color='Local', #El color depende de las cantidades
                    color_continuous_scale="greens", #greens
                    animation_frame='Año'
                    #scope="north america"
                   )
        fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
    elif value == 'Trasbordo':
        fig = px.choropleth(data_frame=df, 
                    geojson=geo_pan, 
                    locations='PROV_NAME', # nombre de la columna del Dataframe
                    featureidkey='properties.PROV_NAME',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                    color='Trasbordo', #El color depende de las cantidades
                    color_continuous_scale="greens", #greens
                    animation_frame='Año'
                    #scope="north america"
                   )
        fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
    else:
        fig = px.choropleth(data_frame=df, 
                    geojson=geo_pan, 
                    locations='PROV_NAME', # nombre de la columna del Dataframe
                    featureidkey='properties.PROV_NAME',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                    color='Zona Libre', #El color depende de las cantidades
                    color_continuous_scale="greens", #greens
                    animation_frame='Año'
                    #scope="north america"
                   )
        fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
    return fig
    

In [80]:
if __name__ == ('__main__'):
    app.run_server(port=5003)
#df = pd.read_csv("D:\CATALAG\Archivos Semestral\PSFC.csv")
#df.head()

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

Dash is running on http://127.0.0.1:5003/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET /assets/s1.css?m=1627386984.0 HTTP/1.1" 304 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET /assets/stylesheet.css?m=1657734382.2705944 HTTP/1.1" 304 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET /_favicon.ico?v=2.5.1 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET /assets/contenedor.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:47:56] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:47:57] "POST /_dash-update-component HTTP/1.1" 200 

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\dash\_callback.py", line 151, in add_context
    output_value = func(*func_arg

127.0.0.1 - - [17/Jul/2022 12:48:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Jul/2022 12:48:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:48:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:48:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:48:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:48:29] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\dash\_callback.py", line 151, in add_context
    output_value = func(*func_arg

127.0.0.1 - - [17/Jul/2022 12:49:24] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Jul/2022 12:49:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:49:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:49:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:49:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:49:28] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\dash\dash.py", line 1472, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "c:\Users\jayso\anaconda3\envs\PFT\lib\site-packages\dash\_callback.py", line 151, in add_context
    output_value = func(*func_arg

127.0.0.1 - - [17/Jul/2022 12:49:28] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Jul/2022 12:49:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:49:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2022 12:49:30] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
df.head("Bocas Fruit Co. Mov. De Contenedores Trasbordo")

TypeError: cannot do positional indexing on RangeIndex with these indexers [Bocas Fruit Co. Mov. De Contenedores Trasbordo] of type str

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
ax.set_title('Provincias de Panama',pad = 20)
df.plot(column="name_1",cmap = "autumn_r",ax=ax,zorder = 100)

Dash is running on http://127.0.0.1:5003/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jul/2022 12:12:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:12:20] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_5_1m1657670236.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:12:20] "GET /_dash-component-suites/dash/deps/react@16.v2_5_1m1657670236.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:12:20] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_5_1m1657670236.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:12:20] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_5_1m1657670236.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:12:20] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_5_1m1657670236.min.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:12:20] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_5_0m1657670236.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 12:12:20] "GET /_dash-component-suites/d